# Traffic Violation RAG System
In this exam, you will implement a Retrieval-Augmented Generation (RAG) system that uses a language model and a vector database to answer questions about traffic violations. The goal is to generate answers with relevant data based on a dataset of traffic violations and fines.

Here are helpful resources:
* [LangChain](https://www.langchain.com/)
* [groq cloud documentation](https://console.groq.com/docs/models)
* [LangChain HuggingFace](https://python.langchain.com/docs/integrations/text_embedding/sentence_transformers/)
* [Chroma Vector Store](https://python.langchain.com/docs/integrations/vectorstores/chroma/)
* [Chroma Website](https://docs.trychroma.com/getting-started)
* [ChatGroq LangChain](https://python.langchain.com/docs/integrations/chat/groq/)
* [LLM Chain](https://api.python.langchain.com/en/latest/chains/langchain.chains.llm.LLMChain.html#langchain.chains.llm.LLMChain)

Dataset [source](https://www.moi.gov.sa/wps/portal/Home/sectors/publicsecurity/traffic/contents/!ut/p/z0/04_Sj9CPykssy0xPLMnMz0vMAfIjo8ziDTxNTDwMTYy83V0CTQ0cA71d_T1djI0MXA30gxOL9L30o_ArApqSmVVYGOWoH5Wcn1eSWlGiH1FSlJiWlpmsagBlKCQWqRrkJmbmqRqUZebngB2gUJAKdERJZmqxfkG2ezgAhzhSyw!!/)

Some installs if needed:
```python
!pip install langchain_huggingface langchain langchain-community langchain_chroma Chroma langchain_groq LLMChain
```

In [1]:
%pip install -qU langchain-huggingface

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 1.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 405.1/405.1 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 249.1/249.1 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.0/290.0 kB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 1.6 MB/s eta 0:00:00


In [2]:
!pip install langchain langchain_community unstructured sentence_transformers tiktoken chromadb langchain_chroma langchain_groq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 11.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 43.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 63.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 58.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 52.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 599.2/599.2 kB 37.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 65.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 273.8/273.8 kB 21.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.6/94.6 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
!kaggle datasets download -d khaledzsa/dataset
!unzip dataset.zip

Dataset URL: https://www.kaggle.com/datasets/khaledzsa/dataset
License(s): unknown
  0% 0.00/3.73k [00:00<?, ?B/s]
100% 3.73k/3.73k [00:00<00:00, 8.01MB/s]
Archive:  dataset.zip
  inflating: Dataset.csv             


## Step 1: Install Required Libraries

To begin, install the necessary libraries for this project. The libraries include `LangChain` for building language model chains, and `Chroma` for managing a vector database.

In [4]:
import os
import pandas as pd
import langchain_huggingface
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
import markdown
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain_groq import ChatGroq

In [5]:
from langchain_community.embeddings.sentence_transformer import (
    SentenceTransformerEmbeddings,
)

/usr/local/lib/python3.10/dist-packages/pydantic/_internal/_fields.py:132: UserWarning: Field "model_name" in HuggingFaceInferenceAPIEmbeddings has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(


# Step 2: Load the Traffic Violations Dataset

You are provided with a dataset of traffic violations. Load the CSV file into a pandas DataFrame and preview the first few rows of the dataset using `.head()`. You can also try and see the dataset's characteristics.

In [6]:
df = pd.read_csv('/content/Dataset.csv')

In [7]:
df

,المخالفة,الغرامة
0,قيادة المركبة في الأسواق التي لا يسمح بالقيادة...,الغرامة المالية 100 - 150 ريال
1,ترك المركبة مفتوحة وفي وضع التشغيل بعد مغادرتها.,الغرامة المالية 100 - 150 ريال
2,عدم وجود تأمين ساري للمركبة.,الغرامة المالية 100 - 150 ريال
3,عبور المشاة للطرق من غير الأماكن المخصصة لهم.,الغرامة المالية 100 - 150 ريال
4,عدم تقيد المشاة بالإشارات الخاصة بهم.,الغرامة المالية 100 - 150 ريال
...,...,...
99,القيام بأعمال الطرق قبل التنسيق مع الإدارة الم...,الغرامة المالية 5000 - 10000 ريال
100,سماح أصحاب الحيوانات بعبور حيواناتهم من غير ال...,الغرامة المالية 5000 - 10000 ريال
101,استخدام لوحات غير عائدة للمركبة.,الغرامة المالية 5000 - 10000 ريال
102,طمس رقم هيكل المركبة أو محاولة طمسه.,الغرامة المالية 5000 - 10000 ريال


In [8]:
df['المخالفة']

,المخالفة
0,قيادة المركبة في الأسواق التي لا يسمح بالقيادة...
1,ترك المركبة مفتوحة وفي وضع التشغيل بعد مغادرتها.
2,عدم وجود تأمين ساري للمركبة.
3,عبور المشاة للطرق من غير الأماكن المخصصة لهم.
4,عدم تقيد المشاة بالإشارات الخاصة بهم.
...,...
99,القيام بأعمال الطرق قبل التنسيق مع الإدارة الم...
100,سماح أصحاب الحيوانات بعبور حيواناتهم من غير ال...
101,استخدام لوحات غير عائدة للمركبة.
102,طمس رقم هيكل المركبة أو محاولة طمسه.


## Step 3: Create Markdown Content from the Dataset

For each traffic violation in the dataset, you will generate markdown text that describes the violation and the associated fine. Create a loop to iterate through the dataset and store the generated markdown in a list. Each fine should look like this:

**المخالفة** - الغرامة

In [9]:
len(df)

104

In [10]:
df['المخالفة'][1]

'ترك المركبة مفتوحة وفي وضع التشغيل بعد مغادرتها.'

In [11]:
path = './markdown'
os.makedirs(path , exist_ok = True)

In [12]:
palanteis = df['المخالفة']
fines = df['الغرامة']
for i in range(len(df)):
  fine  = fines.iloc[i]
  palanty = palanteis.iloc[i]
  markdown_text = f'#{fine}\n\n'
  markdown_text += f'{palanty}'

  with open(f'{path}/ru{i}.md' , 'w' , encoding = 'utf-8') as file:
    file.write(markdown_text)


In [13]:
marked_text =[]
for fileName in os.listdir(path):
  if fileName.endswith('.md'):
    with open(os.path.join(path,fileName) , 'r' , encoding = 'utf-8') as file:
          temp = file.read()
          html_content = markdown.markdown(temp)
          marked_text.append(temp)


In [14]:
marked_text

['#الغرامة المالية 1000 - 2000 ريال\n\nالوقوف على خطوط السكة الحديدية.',
 '#الغرامة المالية 150 - 300 ريال\n\nعدم ربط حزام الأمان.',
 '#الغرامة المالية 3000 - 6000 ريال\n\nعدم الوقوف عند مراكز الضبط الأمني أو نقاط التفتيش أو عدم الوقوف للدورية الأمنية عند وجود توجيه أو علامة توجب الوقوف.',
 '#الغرامة المالية 5000 - 10000 ريال\n\nسماح أصحاب الحيوانات بعبور حيواناتهم من غير الأماكن المخصصة لها، أو دون التنسيق مع الجهات المختصة.',
 '#الغرامة المالية 1000 - 2000 ريال\n\nعدم إنهاء إجراءات تعديل مجال استعمال المركبة.',
 '#الغرامة المالية 1000 - 2000 ريال\n\nسير المركبة بلا لوحة أمامية.',
 '#الغرامة المالية 1000 - 2000 ريال\n\nالقيادة على أكتاف الطريق وعلى الأرصفة أو المسارات التي يمنع القيادة فيها.',
 '#الغرامة المالية 300 - 500 ريال\n\nعدم التقيد بتنظيمات السير عند تقاطعات الطرق.',
 '#الغرامة المالية 500 - 900 ريال\n\nعدم قيام السائق في حال تغيير اتجاهه بالدوران للخلف بإعطاء أفضلية المرور للمركبات القادمة من الاتجاهات الأخرى.',
 '#الغرامة المالية 500 - 900 ريال\n\nتظليل زجاج المركبة دون الت

## Step 4: Chunk the Markdown Data

Using LangChain's `RecursiveCharacterTextSplitter`, split the markdown texts into smaller chunks that will be stored in the vector database.

In [15]:
splitter = RecursiveCharacterTextSplitter(chunk_size=100, chunk_overlap=30)
documents = splitter.create_documents(marked_text)

## Step 5: Generate Embeddings for the Documents

Generate embeddings for the chunks of text using HuggingFace's pre-trained Arabic language model. These embeddings will be stored in a `Chroma` vector store.

In [16]:
from langchain_huggingface import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name="Omartificial-Intelligence-Space/GATE-AraBert-v0")
arabic_VDB = Chroma.from_documents(documents, embeddings, persist_directory="./chroma_db")

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/205 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/7.50k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/2.02k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/761k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.78M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

# Step 6: Define the RAG Prompt Template

Define a custom prompt template in Arabic to retrieve traffic violation-related answers based on the context. Ensure the template encourages the model to give **advice** in **Arabic**, staying within the context provided.

In [17]:
prompt = '''
 * يتجب عليك الرد باللغة العربية
 * لا تخرج عن النص ولا السياق مهما حدث
 * اعطي نصيحة بسيطة متعلقة بالنص
 * قل قصيدة عن الالتزام في النظام
Context: {context}
Question: {question}

اكتب اجابتك هنا:
'''
arabic_prompt = PromptTemplate(
    template=prompt, input_variables= ["context", "question"]
)

## Step 7: Initialize the Language Model

Initialize the language model using the Groq API. Set up the model with a specific configuration, including the API key, temperature setting, and model name.

In [18]:
key_API = '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'

In [19]:
LLM = ChatGroq(temperature=0.6, groq_api_key=key_API, model_name="llama3-70b-8192")

## Step 8: Create the LLM Chain

Now, you will create an LLM Chain that combines the language model and the prompt template you defined. This chain will be used to generate responses based on the retrieved context.

In [20]:
MODEL = LLMChain(llm=LLM ,prompt=arabic_prompt,verbose=True)

<ipython-input-20-ff39a8bb1ed6>:1: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  MODEL = LLMChain(llm=LLM ,prompt=arabic_prompt,verbose=True)


## Step 9: Implement the Query Function

Create a function `query_rag` that will take a user query as input, retrieve relevant context from the vector store, and use the language model to generate a response based on that context.

In [21]:
def query_rag(query: str , threshold : float):
    results = arabic_VDB.similarity_search_with_score(query, k=4)
    context_text = "\n\n".join([doc.page_content for doc, score in results if score > threshold])
    rag_response = MODEL.invoke({"context": context_text, "question": query})
    return rag_response , results

## Step 10: Inference - Running Queries in the RAG System

In this final step, you will implement an inference pipeline to handle real-time queries. You will allow the system to retrieve the most relevant violations and fines based on a user's input and generate a response.

1. Inference Workflow:

  * The user inputs a query (e.g., "ماهي الغرامة على القيادة بدون رخصة؟").
  * The system searches for the most relevant context from the traffic violation vector store.
  * It generates an answer and advice based on the context.

2. Goal:
  * Run the inference to answer questions based on the traffic violation dataset.

In [22]:
res ,docs = query_rag( 'ماهي الغرامة على القيادة بدون رخصة؟' , 0.5)
res

Prompt after formatting:

 * يتجب عليك الرد باللغة العربية
 * لا تخرج عن النص ولا السياق مهما حدث
 * اعطي نصيحة بسيطة متعلقة بالنص 
 * قل قصيدة عن الالتزام في النظام
Context: #الغرامة المالية 150 - 300 ريال

عدم حمل رخصة القيادة أو رخصة السير أثناء القيادة.

#الغرامة المالية 500 - 900 ريال

قيادة المركبة داخل الأنفاق من غير إضاءة أنوارها.

قيادة المركبة برخصة قيادة لا تتناسب مع حجم المركبة ونوع استخدامها.

#الغرامة المالية 3000 - 6000 ريال

سير المركبة بلا لوحة خلفية، أو بلا لوحات.
Question: ماهي الغرامة على القيادة بدون رخصة؟

اكتب اجابتك هنا:


> Finished chain.


{'context': '#الغرامة المالية 150 - 300 ريال\n\nعدم حمل رخصة القيادة أو رخصة السير أثناء القيادة.\n\n#الغرامة المالية 500 - 900 ريال\n\nقيادة المركبة داخل الأنفاق من غير إضاءة أنوارها.\n\nقيادة المركبة برخصة قيادة لا تتناسب مع حجم المركبة ونوع استخدامها.\n\n#الغرامة المالية 3000 - 6000 ريال\n\nسير المركبة بلا لوحة خلفية، أو بلا لوحات.',
 'question': 'ماهي الغرامة على القيادة بدون رخصة؟',
 'text': 'الغرامة على القيادة بدون رخصة هي 150 - 300 ريال.\n\nنصيحة بسيطة: دائماً احتفظ برخصتك معك أثناء القيادة لتجنب دفع الغرامات المالية.\n\nقصيدة عن الالتزام في النظام:\n\nفي طريقنا ن चलत فلا ننسى\nرخصتنا معنا دائماً ن携ها\nلألا نكون من المخالفين\nونعاقب بالغرامات المالية\n\nفلتكن رخصتنا رفيقنا\nفي كل مكان نذهب إليه\nونحترم قوانين السير\nلنكون آمنين دائماً.'}

In [23]:
docs

[(Document(metadata={}, page_content='#الغرامة المالية 150 - 300 ريال\n\nعدم حمل رخصة القيادة أو رخصة السير أثناء القيادة.'),
  301.68902587890625),
 (Document(metadata={}, page_content='#الغرامة المالية 500 - 900 ريال\n\nقيادة المركبة داخل الأنفاق من غير إضاءة أنوارها.'),
  324.84656150079104),
 (Document(metadata={}, page_content='قيادة المركبة برخصة قيادة لا تتناسب مع حجم المركبة ونوع استخدامها.'),
  327.87420136588736),
 (Document(metadata={}, page_content='#الغرامة المالية 3000 - 6000 ريال\n\nسير المركبة بلا لوحة خلفية، أو بلا لوحات.'),
  344.46240234375)]

In [ ]:
res ,docs = query_rag( 'ماهي الغرامة على القيادة بدون رخصة؟' , 0.5)
res